In [1]:
%load_ext autoreload
%autoreload 2

#SFTP 

# # Use
# cnopts = pysftp.CnOpts()
# cnopts.hostkeys = None


# #Get Data from PS Data Export Manager SFTP folder, and move to BigQuery
# with pysftp.Connection(
#     host="ps.com",
#     username="greendot",
#     password="*********",
#     cnopts=cnopts
# ) as sftp:
#     # Download a remote file to the local machine
#     remote_file = "/greendottn/custom_report_standards_2024"
#     local_file = "local_file.csv"   (This can be dynamic if we want to preserve the file everytime)
#     sftp.get(remote_file, local_file)

# ----------------------------------------------------------

from modules.buckets import create_bucket, upload_to_bucket, download_from_bucket, upload_to_bq_table
import os
import pandas_gbq
import pandas as pd


# Set the GOOGLE_APPLICATION_CREDENTIALS environment variable in order to interact
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'greendotdataflow-848329b50f47.json'



class Create:

    def __init__(self, bucket, end_file_name, local_file_name, project_id, db, table_name, SQL_query, location=None):
        
        self.location = location
        self.bucket = bucket
        self.end_file_name = end_file_name
        self.local_file_name = local_file_name
        self.project_id = project_id
        self.db = db
        self.table_name = table_name
        self.SQL_query = SQL_query


    def process(self):
    
        #Create the bucket, and upload to that bucket. If already created, bypass
        create_bucket(self.bucket, self.location)

        #Upload file to bucket, demonstrates if overwritten or newfile. 
        #End File Name,  Local File Path, Bucket Name
        upload_to_bucket(self.end_file_name , self.local_file_name, self.bucket)


        upload_to_bq_table(cloud_storage_uri = f'gs://{self.bucket}/{self.end_file_name}',
                        project_id = self.project_id,
                        db = self.db,
                        table_name = self.table_name,
                        location = self.location)

        try:
            print('Running query')
            query = pandas_gbq.read_gbq(self.SQL_query, project_id=self.project_id, location=self.location)
            print('Query completed')
        except:
            print('Unable to run query')


        return(query)





already_created_update = Create(location = 'us-south1',
                                bucket='psholdingbucket7',
                                end_file_name = 'students.csv',
                                local_file_name = 'Student_Records.csv',
                                project_id='greendotdataflow',
                                db = 'Students',
                                table_name = 'Student_Records_7',

                                SQL_query='''

                                SELECT * FROM greendotdataflow.Students.Student_Records_7

                                '''  
                    )

brand_new_creation = Create(location = 'us-south1',
                            bucket='psholdingbucket12',
                            end_file_name = 'students.csv',
                            local_file_name = 'Student_Records.csv',
                            project_id='greendotdataflow',
                            db = 'Students',
                            table_name = 'Student_Records_12',

                            SQL_query='''

                            SELECT * FROM greendotdataflow.Students.Student_Records_12

                            '''  
                    )


query_one = already_created_update.process()

query_two = brand_new_creation.process()


#-------------Next step put file in clevers SFTP--------------



Bucket psholdingbucket7 already exists.
File students.csv in bucket psholdingbucket7 was overwritten by local file Student_Records.csv.
Table greendotdataflow.Students.Student_Records_7 already exists, appending new data


100%|██████████| 1/1 [00:00<?, ?it/s]


Running query
Downloading: 100%|██████████|
Query completed


Bucket psholdingbucket12 created in us-south1 with <module 'google.cloud.storage' from 'c:\\Users\\samuel.taylor\\Anaconda3\\lib\\site-packages\\google\\cloud\\storage\\__init__.py'> storage class.
Local File Student_Records.csv was uploaded as a new file students.csv in the bucket psholdingbucket12.
Table greendotdataflow.Students.Student_Records_12 has been created, and data has been sent


100%|██████████| 1/1 [00:00<?, ?it/s]


Running query
Downloading: 100%|██████████|
Query completed


In [2]:
query_one

,student_id,grades,class
0,1,90,Math
1,5,88,Math
2,9,87,Math
3,4,78,English
4,7,82,English
...,...,...,...
145,6,95,Science
146,6,95,Science
147,6,95,Science
148,6,95,Science


In [3]:
query_two

,student_id,grades,class
0,1,90,Math
1,5,88,Math
2,9,87,Math
3,4,78,English
4,7,82,English
5,2,85,History
6,8,91,History
7,10,89,History
8,3,92,Science
9,6,95,Science
